In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TemplateHaskell #-}

import AdventOfCode
import Control.Lens

import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Map.Strict as M
import qualified Data.Vector as V

input <- dayLines 18

In [2]:
type Reg = Char
type ValueReg = Either Int Reg

parseReg = A.satisfy A.isAlpha_ascii
parseValue = A.signed A.decimal

parseValueReg :: A.Parser ValueReg
parseValueReg = A.eitherP parseValue parseReg

In [3]:
data Instr 
    = Snd ValueReg 
    | Set Reg ValueReg
    | Add Reg ValueReg
    | Mul Reg ValueReg
    | Mod Reg ValueReg
    | Rcv Reg
    | Jgz ValueReg ValueReg
    deriving (Eq, Show)

parseInstr = A.choice
    [ Snd <$> ("snd" *> pVR)
    , Set <$> ("set" *> pR)  <*> pVR
    , Add <$> ("add" *> pR)  <*> pVR
    , Mul <$> ("mul" *> pR)  <*> pVR
    , Mod <$> ("mod" *> pR)  <*> pVR
    , Rcv <$> ("rcv" *> pR)
    , Jgz <$> ("jgz" *> pVR) <*> pVR
    ]
    where
        pVR = A.space *> parseValueReg
        pR  = A.space *> parseReg

registerMap = M.fromList $ zip ['a'..'p'] (repeat 0)
instrVec    = V.fromList $ map (parsed parseInstr) input

In [4]:
data MachineState = MachineState
    { _registers :: M.Map Char Int
    , _counter   :: Int
    } deriving (Eq, Show)

makeLenses ''MachineState

getValueReg registers valueReg = case valueReg of
    Left v -> v
    Right r -> registers M.! r

executeInstr instr ms = case instr of
    Snd     vr -> ms & counter +~ 1
    Set reg vr -> ms & counter +~ 1 & registers %~ (\rs -> M.insert reg       (getValueReg rs vr)      rs)
    Add reg vr -> ms & counter +~ 1 & registers %~ (\rs -> M.adjust         (+(getValueReg rs vr)) reg rs)
    Mul reg vr -> ms & counter +~ 1 & registers %~ (\rs -> M.adjust         (*(getValueReg rs vr)) reg rs)
    Mod reg vr -> ms & counter +~ 1 & registers %~ (\rs -> M.adjust     (`mod`(getValueReg rs vr)) reg rs)
    Jgz vr vr' -> if getValueReg (ms ^. registers) vr > 0
        then ms & counter %~ (+ (getValueReg (ms ^. registers) vr'))
        else ms & counter +~ 1
    other -> error $ "don't know how to execute " ++ show other

In [5]:
loop instrs ms lastFreq | (ms ^. counter) > V.length instrs || (ms ^. counter) < 0 = lastFreq
loop instrs ms lastFreq = case instrs V.! (ms ^. counter) of
    Snd vr -> loop instrs (executeInstr (Snd vr) ms) (getValueReg (ms ^. registers) vr)
    Rcv r  -> if ((ms ^. registers) M.! r) /= 0 then lastFreq else loop instrs (ms & counter +~ 1) lastFreq
    curr   -> loop instrs (executeInstr curr ms) lastFreq

In [6]:
loop instrVec (MachineState registerMap 0) 0

3423

In [7]:
data LockedState = Running | Waiting deriving (Eq, Show)
data ConcMachineState = ConcMachineState
    { _ms          :: MachineState
    , _mailbox     :: [Int]
    , _lockedState :: LockedState
    }

makeLenses ''ConcMachineState

send msg cms = cms & mailbox %~ (++ [msg])

executeInstr' instr cms = case instr of
    Rcv reg -> if (cms ^. mailbox) == [] 
        then cms & lockedState .~ Waiting
        else cms 
                & ms %~ (\m -> m & counter +~ 1 & registers %~ (M.insert reg (cms ^. mailbox . to head))) 
                & mailbox %~ tail 
                & lockedState .~ Running
    _ -> cms & ms %~ (executeInstr instr)

loop' instrs cms0 cms1 count
    | cms0  ^. lockedState == Waiting && cms1 ^. lockedState == Waiting    = count
    | cms0 ^. ms ^. counter > V.length instrs || cms0 ^. ms ^. counter < 0 = count
    | cms1 ^. ms ^. counter > V.length instrs || cms1 ^. ms ^. counter < 0 = count
    | Snd vr <- instrs V.! (cms0 ^. ms ^. counter) = let
        value = getValueReg (cms0 ^. ms ^. registers)  vr
        cms1' = send value cms1
        cms0' = executeInstr' (Snd vr) cms0
        in loop' instrs cms0' cms1' count
    | Snd vr <- instrs V.! (cms1 ^. ms ^. counter) = let
        value = getValueReg (cms1 ^. ms ^. registers) vr
        cms0' = send value cms0
        cms1' = executeInstr' (Snd vr) cms1
        in loop' instrs cms0' cms1' (count + 1)
    | otherwise = let
        instr0 = instrs V.! (cms0 ^. ms ^. counter)
        instr1 = instrs V.! (cms1 ^. ms ^. counter)
        cms0' = executeInstr' instr0 cms0
        cms1' = executeInstr' instr1 cms1
        in loop' instrs cms0' cms1' count

initialMs0 = ConcMachineState (MachineState                 registerMap  0) [] Running
initialMs1 = ConcMachineState (MachineState (M.insert 'p' 1 registerMap) 0) [] Running

In [8]:
loop' instrVec initialMs0 initialMs1 0

7493